In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../sp500_ohlcv_2005_2025_2.csv")

In [ ]:

# Suppose df a les colonnes : Date, Ticker, Open, High, Low, Close, Volume
df = df.sort_values(["Ticker", "Date"])

# --- 1) Log-return
df["log_return"] = np.log(df["Close"]) - np.log(df["Close"].shift(1))

# --- 2) Target direction (r_{t+1} > 0)
df["target"] = (df.groupby("Ticker")["log_return"].shift(-1) > 0).astype(int)

# --- 3) Features momentum
df["mom_5"]  = df.groupby("Ticker")["Close"].transform(lambda x: x / x.shift(5) - 1)
df["mom_21"] = df.groupby("Ticker")["Close"].transform(lambda x: x / x.shift(21) - 1)

# --- 4) Features volatilité
df["vol_5"]  = df.groupby("Ticker")["log_return"].transform(lambda x: x.rolling(5).std())
df["vol_21"] = df.groupby("Ticker")["log_return"].transform(lambda x: x.rolling(21).std())

# --- 5) High–Low range
df["range"] = (df["High"] - df["Low"]) / df["Open"]

# --- 6) Volume z-score
df["volume_z"] = df.groupby("Ticker")["Volume"].transform(
    lambda x: (x - x.mean()) / x.std()
)

df['Return'] = df.groupby('Ticker')['Close'].pct_change(fill_method=None)

# --- 7) Clean
df = df.dropna()


In [ ]:
import requests

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0"}

resp = requests.get(url, headers=headers)
resp.raise_for_status()  # lève une erreur si 4xx/5xx

tables = pd.read_html(resp.text, header=0)
sp500 = tables[0]

sp500 = sp500.rename(columns={"Symbol": "Ticker", "GICS Sector": "Sector"})
sp500['Ticker'] = sp500['Ticker'].str.replace('.', '-', regex=False)

df = df.sort_values(by=['Ticker', 'Date'])

df = df.merge(
    sp500[['Ticker', 'Sector']], 
    on='Ticker', 
    how='left'
)

df.tail()

la variable Sector est catégorielle par conséquent on va l'encoder par par la volatilité moyenne du secteur. Ce choix est motivé par la raison suivante. On veut faire comprendre au modèle que certains secteurs sont très volatils et très dépendants de chocs extérieur ainsi pour ces secteurs la, l'évolution futur de l'action est moins évidente.

In [ ]:
# 1) Calcul de volatilité moyenne par secteur
sector_vol_mean = df.groupby("Sector")["vol_21"].mean()

# 2) Encodage de Sector par cette moyenne
df["Sector_encoded"] = df["Sector"].map(sector_vol_mean)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report



def regression(df, features):

    # --- 8) Train/val/test split
    train = df[df["Date"] < "2018-01-01"]
    val   = df[(df["Date"] >= "2018-01-01") & (df["Date"] < "2021-01-01")]
    test  = df[df["Date"] >= "2021-01-01"].copy()   # copy important

    X_train, y_train = train[features], train["target"]
    X_val, y_val     = val[features], val["target"]
    X_test, y_test   = test[features], test["target"]
    
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Prédictions sur le test
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)


    # --- Autres métriques globales
    print("Accuracy :", accuracy_score(y_test, y_pred))

    test["pred"] = y_pred

    # --- Calcul F1 par secteur (ajuste "Sector" si ta colonne s'appelle autrement)
    sector_f1 = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    return  model, sector_f1


In [ ]:
regression(df, ["mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"])


De premier abord, la regression parait ne pas être suffisante pour prédire le signe du rendement

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix, classification_report
import xgboost as xgb
import matplotlib.pyplot as plt

def training(df, features):

    # --- 8) Train/val/test split
    train = df[df["Date"] < "2018-01-01"]
    val   = df[(df["Date"] >= "2018-01-01") & (df["Date"] < "2021-01-01")]
    test  = df[df["Date"] >= "2021-01-01"].copy()   # copy important

    X_train, y_train = train[features], train["target"]
    X_val, y_val     = val[features], val["target"]
    X_test, y_test   = test[features], test["target"]
    
    # --- 9) Modèle XGBoost
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Prédictions sur le test
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)


    # --- Autres métriques globales
    print("Accuracy :", accuracy_score(y_test, y_pred))
    # print("F1        :", f1_score(y_test, y_pred))
    # print("AUC       :", roc_auc_score(y_test, y_pred_proba))
    # print()
    # print(confusion_matrix(y_test, y_pred))
    # print()
    # print(classification_report(y_test, y_pred))

     # Ajout des prédictions dans test
    test["pred"] = y_pred

    # --- Calcul F1 par secteur (ajuste "Sector" si ta colonne s'appelle autrement)
    sector_f1 = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    return  sector_f1


In [ ]:
training(df, ["Close", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"])

In [ ]:
training(df, ["mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"])

In [ ]:
training(df, ["mom_5", "vol_5", "vol_21", "range", "volume_z"])


In [ ]:
training(df, ["mom_5", "vol_5", "range", "volume_z"])

In [ ]:
training(df, ["mom_5", "vol_5", "range", "volume_z", "Return"])


In [ ]:
training(df, ["mom_5", "vol_5", "range", "volume_z", "Sector_encoded"])

In [ ]:
training(df, ["mom_5", "vol_5", "volume_z"])

In [ ]:
training(df, ["Open", "Close", "High", "Low", "Volume", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"])

In [ ]:
training(df, ["mom_5", "vol_5", "range", "volume_z"])

In [ ]:
training(df, ["mom_5", "vol_5", "range"])

In [ ]:
training(df, ["mom_5", "vol_5", "range"])

meme en ajoutant des variable pertinentes, l'accuracy reste sensiblement la meme peu importe le secteur. L'accuracy max réalisé est 0.5167. Peut etre qu'étant donné le fait que les années d'entrainement sont très anciennes par rapport aux années de test le modèle est mal adapté aux nouvelles années

Changeons de méthode d'entrainement : éparpillons des années de test et de validation dans tous le dataset

# Ajout de la variable secteur

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix, classification_report
import xgboost as xgb
import matplotlib.pyplot as plt

def training2(df, features):

    # --- 8) Train/val/test split
    mask_train = (
        df["Date"].between("2004-01-01", "2008-01-01") |
        df["Date"].between("2010-01-01", "2013-01-01") |
        df["Date"].between("2015-01-01", "2018-01-01") |
        df["Date"].between("2020-01-01", "2023-01-01")
    )

    mask_val = (
        df["Date"].between("2008-01-01", "2009-01-01") |
        df["Date"].between("2013-01-01", "2014-01-01") |
        df["Date"].between("2018-01-01", "2019-01-01") |
        df["Date"].between("2023-01-01", "2024-01-01")
    )

    mask_test = (
        df["Date"].between("2009-01-01", "2010-01-01") |
        df["Date"].between("2014-01-01", "2015-01-01") |
        df["Date"].between("2019-01-01", "2020-01-01") |
        df["Date"].between("2024-01-01", "2025-01-01")
    )

    train = df[mask_train]
    val   = df[mask_val]
    test  = df[mask_test].copy()   # copy pour pouvoir ajouter des colonnes

    X_train, y_train = train[features], train["target"]
    X_val, y_val     = val[features], val["target"]
    X_test, y_test   = test[features], test["target"]
    
    # --- 9) Modèle XGBoost
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Prédictions sur le test
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Ajout des prédictions dans test
    test["pred"] = y_pred

    # --- Calcul F1 par secteur (ajuste "Sector" si ta colonne s'appelle autrement)
    sector_f1 = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    print("Accuracy :", accuracy_score(y_test, y_pred))

    return  sector_f1
    

In [ ]:
training2(df, ["mom_21", "vol_21", "range", "volume_z", "Sector_encoded"])


In [ ]:
training2(df, ["mom_5", "vol_5", "volume_z", "Sector_encoded"])


In [ ]:
training2(df, ["mom_5", "vol_5", "volume_z"])


In [ ]:
training2(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"])


la meilleur accuracy est meilleure que celle obtenue précédemment 0.5293 mais reste cependant faible. 

Essayons de préciser nos modèles en les concentrant sur un secteur. Peut être que les modèles précédent essayaient de trop généraliser ce qui les rendait imprécis.

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix, classification_report
import xgboost as xgb
import matplotlib.pyplot as plt

def training3(df, features, sector):

    # --- 8) Train/val/test split
    mask_train = (
        (df["Sector"] == sector) &
        (df["Date"].between("2004-01-01", "2008-01-01") |
        df["Date"].between("2010-01-01", "2013-01-01") |
        df["Date"].between("2015-01-01", "2018-01-01") |
        df["Date"].between("2020-01-01", "2023-01-01"))
    )

    mask_val = (
        (df["Sector"] == sector) &
        (df["Date"].between("2008-01-01", "2009-01-01") |
        df["Date"].between("2013-01-01", "2014-01-01") |
        df["Date"].between("2018-01-01", "2019-01-01") |
        df["Date"].between("2023-01-01", "2024-01-01"))
    )

    mask_test = (       
        (df["Sector"] == sector) &
        (df["Date"].between("2009-01-01", "2010-01-01") |
        df["Date"].between("2014-01-01", "2015-01-01") |
        df["Date"].between("2019-01-01", "2020-01-01") |
        df["Date"].between("2024-01-01", "2025-01-01"))
    )

    train = df[mask_train]
    val   = df[mask_val]
    test  = df[mask_test].copy()   # copy pour pouvoir ajouter des colonnes

    X_train, y_train = train[features], train["target"]
    X_val, y_val     = val[features], val["target"]
    X_test, y_test   = test[features], test["target"]
    
    # --- 9) Modèle XGBoost
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Prédictions sur le test
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Ajout des prédictions dans test
    test["pred"] = y_pred

    # --- Calcul F1 par secteur (ajuste "Sector" si ta colonne s'appelle autrement)
    sector_f1 = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    print("Accuracy :", accuracy_score(y_test, y_pred))

    return  sector_f1
    

In [ ]:
training3(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "Financials")

In [ ]:
training3(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "Utilities")

In [ ]:
training3(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "Information Technology")

In [ ]:
def training4(df, features, ticker):

    # --- 8) Train/val/test split
    mask_train = (
        (df["Ticker"] == ticker) &
        (df["Date"].between("2004-01-01", "2008-01-01") |
        df["Date"].between("2010-01-01", "2013-01-01") |
        df["Date"].between("2015-01-01", "2018-01-01") |
        df["Date"].between("2020-01-01", "2023-01-01"))
    )

    mask_val = (
        (df["Ticker"] == ticker) &
        (df["Date"].between("2008-01-01", "2009-01-01") |
        df["Date"].between("2013-01-01", "2014-01-01") |
        df["Date"].between("2018-01-01", "2019-01-01") |
        df["Date"].between("2023-01-01", "2024-01-01"))
    )

    mask_test = (       
        (df["Ticker"] == ticker) &
        (df["Date"].between("2009-01-01", "2010-01-01") |
        df["Date"].between("2014-01-01", "2015-01-01") |
        df["Date"].between("2019-01-01", "2020-01-01") |
        df["Date"].between("2024-01-01", "2025-01-01"))
    )

    train = df[mask_train]
    val   = df[mask_val]
    test  = df[mask_test].copy()   # copy pour pouvoir ajouter des colonnes

    X_train, y_train = train[features], train["target"]
    X_val, y_val     = val[features], val["target"]
    X_test, y_test   = test[features], test["target"]
    
    # --- 9) Modèle XGBoost
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Prédictions sur le test
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Ajout des prédictions dans test
    test["pred"] = y_pred

    # --- Calcul F1 par secteur (ajuste "Sector" si ta colonne s'appelle autrement)
    sector_f1 = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    print("Accuracy :", accuracy_score(y_test, y_pred))

    return  sector_f1

In [ ]:
training4(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "GOOGL")

In [ ]:
training4(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "XOM")


In [ ]:
training4(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "JPM")


In [ ]:
training4(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "PLD")


On remarque que souvent la catégories la moins bien prédites est celle de l'énergie qui est le secteur le plus sensible au choc d'après ce qui a été vu précédemment. Peut être que le faible score provient des période de crises dues à des chocs extérieurs aux marché qui sont imprévisibles à l'aide 'uniquement les données. Essayons de se limiter à la période la plus calme de 2005-2025

In [ ]:
def training5(df, features):

    # --- 8) Train/val/test split
    mask_train = (
        df["Date"].between("2010-01-01", "2012-01-01") 
    )

    mask_val = (
        df["Date"].between("2012-01-01", "2013-01-01") 
    )

    mask_test = (
        df["Date"].between("2013-01-01", "2014-01-01") 
    )

    train = df[mask_train]
    val   = df[mask_val]
    test  = df[mask_test].copy()   # copy pour pouvoir ajouter des colonnes

    X_train, y_train = train[features], train["target"]
    X_val, y_val     = val[features], val["target"]
    X_test, y_test   = test[features], test["target"]
    
    # --- 9) Modèle XGBoost
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Prédictions sur le test
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Ajout des prédictions dans test
    test["pred"] = y_pred

    # --- Calcul F1 par secteur (ajuste "Sector" si ta colonne s'appelle autrement)
    sector_f1 = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    print("Accuracy :", accuracy_score(y_test, y_pred))

    return  sector_f1
    

In [ ]:
training5(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"])


Malheuresement les résultats ne sont pas sensiblement meilleur

 Or on a vu précédemment que ces périodes de crise peuvent être caractérisées par des période de volatilité élevée. Ainsi on va retirer du dataset tous les moments ou la volatilité est élevée statistiquement.

In [ ]:
def remove_outliers_iqr_per_ticker(df):
    # Fonction appliquée à chaque groupe (chaque Ticker)
    def filter_group(g):
        q1 = g['vol_21'].quantile(0.25)
        q3 = g['vol_21'].quantile(0.75)
        iqr = q3 - q1
        upper = q3 + 3* iqr

        # on garde seulement les lignes sous la borne haute
        return g[g['vol_21'] <= upper]

    # groupby puis concaténation automatique
    return df.groupby('Ticker', group_keys=False).apply(filter_group, include_groups=False)

In [ ]:
def training6(df, features):

    # --- 8) Train/val/test split
    mask_train = (
        df["Date"].between("2004-01-01", "2008-01-01") |
        df["Date"].between("2010-01-01", "2013-01-01") |
        df["Date"].between("2015-01-01", "2018-01-01") |
        df["Date"].between("2020-01-01", "2023-01-01")
    )

    mask_val = (
        df["Date"].between("2008-01-01", "2009-01-01") |
        df["Date"].between("2013-01-01", "2014-01-01") |
        df["Date"].between("2018-01-01", "2019-01-01") |
        df["Date"].between("2023-01-01", "2024-01-01")
    )

    mask_test = (
        df["Date"].between("2009-01-01", "2010-01-01") |
        df["Date"].between("2014-01-01", "2015-01-01") |
        df["Date"].between("2019-01-01", "2020-01-01") |
        df["Date"].between("2024-01-01", "2025-01-01")
    )

    train_uncleaned = df[mask_train]
    val_uncleaned   = df[mask_val]
    test_uncleaned  = df[mask_test].copy()   # copy pour pouvoir ajouter des colonnes

    train = remove_outliers_iqr_per_ticker(train_uncleaned)
    val = remove_outliers_iqr_per_ticker(val_uncleaned)
    test = remove_outliers_iqr_per_ticker(test_uncleaned).copy()

    X_train, y_train = train[features], train["target"]
    X_val, y_val     = val[features], val["target"]
    X_test, y_test   = test[features], test["target"]
    
    # --- 9) Modèle XGBoost
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Prédictions sur le test
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Ajout des prédictions dans test
    test["pred"] = y_pred

    # --- Calcul F1 par secteur (ajuste "Sector" si ta colonne s'appelle autrement)
    sector_f1 = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    print("Accuracy :", accuracy_score(y_test, y_pred))

    return  sector_f1
    

In [ ]:
training6(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21", "vol_21"])


In [ ]:
training6(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"])


# Conclusion

Dans un premier temps, nous avons tenté de construire un modèle de machine learning capable de prédire le signe du rendement journalier des actions du S&P 500.
Cette approche s’est révélée insuffisante : malgré l’utilisation de différents modèles (régression logistique, XGBoost) et d’un ensemble de variables explicatives dérivées du prix (momentum 5 et 21 jours, volatilité instantanée, amplitude intraday, volume standardisé), les performances prédictives sont restées très proches du hasard.

Ce résultat n’est pas dû à un problème de modélisation ou de qualité des données : il reflète au contraire une propriété structurelle des rendements financiers quotidiens.

En effet, la littérature empirique (Lo & MacKinlay, 1999 ; Bouchaud et al., 2003) montre que les rendements journaliers des actifs financiers sont extrêmement bruités et présentent :

une autocorrélation quasi nulle, ce qui signifie que le rendement d’un jour ne contient presque aucune information exploitable pour prédire celui du lendemain ;

une variance dominée par le bruit de marché, lui-même influencé par des facteurs non observables dans les données OHLCV (annonces macroéconomiques, surprises de résultats, flux d’ordres intraday, microstructure, sentiment, chocs exogènes, etc.) ;

une prédictibilité directionnelle théorique très faible, généralement limitée à 2–4 % d’information exploitable, ce qui entraîne un plafond d’accuracy proche de 52–54 % même pour les modèles les plus puissants.

Dans ce contexte, les variables utilisées — basées sur des caractéristiques de prix relativement lentes (momentum, volatilité historique, volume, amplitudes intraday) — sont structurellement peu informatives pour capturer le signal directionnel à un horizon aussi court. La faible performance observée est donc cohérente avec les limites théoriques du problème.

En revanche, ces mêmes variables sont fortement liées à la dynamique de la volatilité, qui présente une autocorrélation élevée et des régimes persistants (volatility clustering). Contrairement aux rendements, la volatilité est un processus beaucoup plus régulier et prévisible, ce qui en fait une cible de modélisation beaucoup plus appropriée.

C’est pourquoi nous avons choisi, dans un second temps, de réorienter notre travail vers la prédiction de la volatilité future, un problème mieux posé et pour lequel les méthodes d’apprentissage peuvent exploiter un véritable signal.